In [5]:
#!pip install duckdb --quiet

In [6]:
import pandas as pd
import duckdb

In [7]:
dados = [
    
    {"Data": "2021-08-08", "Loja": "A", "Produto": "Cadeira", "Categoria": "Moveis", "Unidades": 3, "Valor":600},
    {"Data": "2021-02-02", "Loja": "A", "Produto": "Notebook", "Categoria": "Eletronico", "Unidades": 1, "Valor":4000},
    {"Data": "2021-02-02", "Loja": "B", "Produto": "Cadeira", "Categoria": "Moveis", "Unidades": 1, "Valor":300},
    {"Data": "2021-03-03", "Loja": "B", "Produto": "Cadeira", "Categoria": "Moveis", "Unidades": 4, "Valor":650},
    {"Data": "2021-02-02", "Loja": "B", "Produto": "Mesa", "Categoria": "Moveis", "Unidades": 1, "Valor":200},
    {"Data": "2021-04-04", "Loja": "B", "Produto": "Mesa", "Categoria": "Moveis", "Unidades": 2, "Valor":400},
    {"Data": "2021-01-02", "Loja": "C", "Produto": "Notebook", "Categoria": "Eletronico", "Unidades": 2, "Valor":4500},
    {"Data": "2021-01-02", "Loja": "C", "Produto": "Smartphone", "Categoria": "Eletronico", "Unidades": 2, "Valor":2000},
    {"Data": "2021-05-05", "Loja": "C", "Produto": "Notebook", "Categoria": "Eletronico", "Unidades": 1, "Valor":5000}
]

df = pd.DataFrame(dados)

display(df)

,Data,Loja,Produto,Categoria,Unidades,Valor
0,2021-08-08,A,Cadeira,Moveis,3,600
1,2021-02-02,A,Notebook,Eletronico,1,4000
2,2021-02-02,B,Cadeira,Moveis,1,300
3,2021-03-03,B,Cadeira,Moveis,4,650
4,2021-02-02,B,Mesa,Moveis,1,200
5,2021-04-04,B,Mesa,Moveis,2,400
6,2021-01-02,C,Notebook,Eletronico,2,4500
7,2021-01-02,C,Smartphone,Eletronico,2,2000
8,2021-05-05,C,Notebook,Eletronico,1,5000


In [8]:
vendas_por_produto = df.groupby("Produto")["Valor"].sum()
display(vendas_por_produto)

vendas_por_loja = df.groupby("Loja")["Valor"].sum()
display(vendas_por_loja)

Produto
Cadeira        1550
Mesa            600
Notebook      13500
Smartphone     2000
Name: Valor, dtype: int64

Loja
A     4600
B     1550
C    11500
Name: Valor, dtype: int64

In [9]:
vendas_por_produtoLoja = df.groupby(["Produto", "Loja"])["Valor"].sum()
display(vendas_por_produtoLoja)

Produto     Loja
Cadeira     A        600
            B        950
Mesa        B        600
Notebook    A       4000
            C       9500
Smartphone  C       2000
Name: Valor, dtype: int64

In [10]:
display(vendas_por_produtoLoja.unstack())

Loja,A,B,C
Produto,,,
Cadeira,600.0,950.0,NaN
Mesa,NaN,600.0,NaN
Notebook,4000.0,NaN,9500.0
Smartphone,NaN,NaN,2000.0


In [11]:
df["Data"] = pd.to_datetime(df["Data"])

In [12]:
df["Ano"] = df["Data"].dt.year
df["Mes"] = df["Data"].dt.month

In [13]:
df.groupby("Mes")["Valor"].sum()

Mes
1    6500
2    4500
3     650
4     400
5    5000
8     600
Name: Valor, dtype: int64

In [14]:
display(df.groupby(["Data", "Loja"])["Valor"].sum().unstack())

Loja,A,B,C
Data,,,
2021-01-02,NaN,NaN,6500.0
2021-02-02,4000.0,500.0,NaN
2021-03-03,NaN,650.0,NaN
2021-04-04,NaN,400.0,NaN
2021-05-05,NaN,NaN,5000.0
2021-08-08,600.0,NaN,NaN


In [15]:
pd.pivot_table(df, values="Valor", index="Loja", 
               columns="Produto", aggfunc="sum", fill_value=0)

Produto,Cadeira,Mesa,Notebook,Smartphone
Loja,,,,
A,600,0,4000,0
B,950,600,0,0
C,0,0,9500,2000


In [21]:
dim_customer = pd.DataFrame({
    "customer_id": [1, 2, 3],
    "Genero": ["M", "F", "F"],
    "Escolaridade": ["graduado", "mestre", "doutor"]
})

dim_tempo = pd.DataFrame({
    "tempo_id": [202306, 202307],
    "Mes": ["Junho", "Julho"],
    "Ano": [2023, 2023]
})

fatos = pd.DataFrame({
    "customer_id": [1, 2, 1, 3, 2],
    "tempo_id": [202306, 202306, 20206, 202307, 202307],
    "valorDaVenda": [1000, 2000, 1324, 49204, 2931],
})

duckdb.register("fatos", fatos)
duckdb.register("dim_tempo", dim_tempo)
duckdb.register("dim_customer", dim_customer)

duckdb.query(""" 
SELECT
    c.Escolaridade,
    t.Mes,
    
    ROUND(AVG(f.valorDavenda), 2) AS media,
    SUM(f.valorDavenda) as soma
FROM fatos f
JOIN dim_customer c ON f.customer_id = c.customer_id
JOIN dim_tempo t ON f.tempo_id = t.tempo_id
GROUP BY CUBE (c.Escolaridade, t.Mes)
ORDER BY c.Escolaridade NULLS LAST, t.Mes NULLS LAST
""").df()

,Escolaridade,Mes,media,soma
0,doutor,Julho,49204.00,49204.0
1,doutor,None,49204.00,49204.0
2,graduado,Junho,1000.00,1000.0
3,graduado,None,1000.00,1000.0
4,mestre,Julho,2931.00,2931.0
5,mestre,Junho,2000.00,2000.0
6,mestre,None,2465.50,4931.0
7,None,Julho,26067.50,52135.0
8,None,Junho,1500.00,3000.0
9,None,None,13783.75,55135.0
